<a href="https://colab.research.google.com/github/RyuichiSaito1/covid19-twitter-usa-restoring/blob/main/roberta_large_fine_tuning_accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import auth
auth.authenticate_user()

In [ ]:
!pip install transformers
!pip install torch
!pip install accelerate
!pip install --upgrade accelerate
# After installing, restart the runtime.

In [ ]:
import torch

class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
import pandas as pd
from torch.utils.data import DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report, confusion_matrix

# Function to read data from TSV file using pandas
def read_tsv(file_path):
    data = pd.read_table(file_path, names=['content', 'label'], dtype='object', engine='python')
    return data

# Test data file path (Replace with your Google Drive directory and file)
file_path = '/content/drive/My Drive/covid-twitter-usa-normal/data/training_data/gpt-3.5/test_data_2021_shuffle_majority_vote_gpt3.5.tsv'

# Read data from TSV file using pandas
test_data = read_tsv(file_path)

# Initialize the tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')

# Encode the test data
test_encodings = tokenizer(test_data['content'].tolist(), truncation=True, padding=True)

# Convert the string labels to integers
test_labels = [int(label) for label in test_data['label']]

# Create the test dataset
test_dataset = TweetDataset(test_encodings, test_labels)

# Initialize the model
model = RobertaForSequenceClassification.from_pretrained('/content/drive/MyDrive/covid-twitter-usa-normal/models/roberta-large/results/checkpoint-339')

# Create a DataLoader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=16)

# Lists to store true and predicted labels
true_labels = []
predicted_labels = []

# Use the model to predict the labels of the test data
for batch in test_loader:
    inputs = {key: val.to(model.device) for key, val in batch.items() if key != 'labels'}
    labels = batch['labels'].to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    true_labels.extend(labels.tolist())
    predicted_labels.extend(predictions.tolist())

# Calculate and display accuracy, recall, precision, and F1 score
accuracy = accuracy_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels, average=None)
precision = precision_score(true_labels, predicted_labels, average=None)
f1 = f1_score(true_labels, predicted_labels, average=None)

# Display classification report and confusion matrix
print("Classification Report:")
print(classification_report(true_labels, predicted_labels))
print("\nConfusion Matrix:")
print(confusion_matrix(true_labels, predicted_labels))

# Display metrics for each class and macro/micro averages
macro_avg = precision.mean(), recall.mean(), f1.mean()
micro_avg = precision.sum() / 3, recall.sum() / 3, f1.sum() / 3

# Display metrics for each class and macro/micro averages
print("+--------------+-----------+----------+----------+----------+")
print("|   Metric     | Accuracy  |  Recall  | Precision|  F1 Score |")
print("+--------------+-----------+----------+----------+----------+")
for i in range(3):
    print(f"| Class {i}      |    {accuracy:.2f}   |   {recall[i]:.2f}   |   {precision[i]:.2f}   |   {f1[i]:.2f}   |")
print("+--------------+-----------+----------+----------+----------+")
print(f"| Macro Average|    {accuracy:.2f}   |   {recall.mean():.2f}   |   {precision.mean():.2f}   |   {f1.mean():.2f}   |")
print("+--------------+-----------+----------+----------+----------+")
print(f"| Micro Average|    {accuracy:.2f}   |   {recall.sum()/3:.2f}   |   {precision.sum()/3:.2f}   |   {f1.sum()/3:.2f}   |")
print("+--------------+-----------+----------+----------+----------+")


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.87      0.82       270
           1       0.85      0.37      0.52       214
           2       0.72      0.94      0.82       265

    accuracy                           0.76       749
   macro avg       0.78      0.73      0.72       749
weighted avg       0.77      0.76      0.73       749


Confusion Matrix:
[[236  10  24]
 [ 62  80  72]
 [ 11   4 250]]
+--------------+-----------+----------+----------+----------+
|   Metric     | Accuracy  |  Recall  | Precision|  F1 Score |
+--------------+-----------+----------+----------+----------+
| Class 0      |    0.76   |   0.87   |   0.76   |   0.82   |
| Class 1      |    0.76   |   0.37   |   0.85   |   0.52   |
| Class 2      |    0.76   |   0.94   |   0.72   |   0.82   |
+--------------+-----------+----------+----------+----------+
| Macro Average|    0.76   |   0.73   |   0.78   |   0.72   |
+--------------+-----------+------